In [1]:
import pandas as pd

In [4]:
from os import getcwd

In [2]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
df_jan = pd.read_parquet\
    (getcwd()+"/yellow_tripdata_2022-01.parquet")
df_feb = pd.read_parquet\
    (getcwd()+"/yellow_tripdata_2022-02.parquet")

In [12]:
# get columns and rows
df_jan.shape

(2463931, 19)

In [16]:
df_feb.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [17]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime

In [20]:
# Standard deviation of the trips duration in Jan 2022 Yellow Tax Trip
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)
df_feb.duration = df_feb.duration.apply(lambda td: td.total_seconds() / 60)
df_jan['duration'].std()

46.44530513776802

In [21]:
df_jan.shape

(2463931, 20)

In [39]:
df_jan_no_outliers = df_jan[(df_jan.duration >= 1)\
                            &(df_jan.duration <= 60)]
df_jan_no_outliers.shape


(2421440, 20)

In [27]:
# How many records left (in January after dropping outliers)
df_jan_no_outliers.shape[0]/df_jan.shape[0]

0.9827547930522406

In [34]:
df_jan_no_outliers.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [45]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_jan_no_outliers[categorical] = df_jan_no_outliers[categorical].astype(str)

/var/folders/gg/27ydp9614p9cpq8pg5r7rklc0000gn/T/ipykernel_14534/3113143432.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_no_outliers[categorical] = df_jan_no_outliers[categorical].astype(str)


In [46]:
# checking dtypes
df_jan_no_outliers.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [31]:
train_dicts = df_jan_no_outliers[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_jan_no_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.001496179375642

In [53]:
X_train.shape
# After OHE, there are 515 columns of encoded categorical data and 1 numerical column

(2421440, 516)

In [47]:
X_train

<2421440x516 sparse matrix of type '<class 'numpy.float64'>'
	with 7264320 stored elements in Compressed Sparse Row format>

In [57]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [58]:
FILE_PATH = getcwd()+"/yellow_tripdata_2022-01.parquet"

In [65]:
df_train = read_dataframe(FILE_PATH)
df_val = read_dataframe(getcwd()+"/yellow_tripdata_2022-02.parquet")

In [67]:
len(df_train),len(df_val)

(2421440, 2918187)

In [88]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [89]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [90]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [91]:
# RMSE on train
mean_squared_error(y_train,y_pred,squared=False)

6.986190814952337

In [92]:
y_pred = lr.predict(X_val)

In [93]:
#RMSE on val
mean_squared_error(y_val,y_pred,squared=False)

7.786407163179794